In [1]:
## Import packages
import numpy as np
import math
import random
from scipy.stats import poisson

## Setup problem definition

## Test case

- T = 4h
- d = 5 min # the size of an interval
- beta = 9 minutes # service time (model by the exponential distribution)
- N = 24 patients
- I = T / d # the number of intervals

In [2]:
class schedule:
  def __init__(self, beta: int, T: int, d: int, N: int):
    self.beta = beta # = 1/μ : average service time
    self.T = T # number of intervals
    self.d = d # length of interval
    self.N = N # total number of patients
    self.x:list[int] = np.repeat(0, self.T) # schedule with x[t] as number of 
                                            # patients scheduled at the start of
                                            # interval t, t = 1,...,T; initially
                                            # set at zero
                                            
  def reset_schedule(self):
    for t in range(self.T):
      self.x[t] = 0

  def make_random_schedule(self, min_x: int, max_x: int, step: int=1):
    n = self.N
    for i in range(0,self.T, step):
      if n >= 0:
        r = random.randint(min_x, max_x)
        self.x[i] = min(r,n)
        n = n - r
        print(n, self.x[i])
      else:
        return
        
  def make_initial_schedule(self):
    for i in range(self.N):
      t = round(i*self.T / self.N)
      if i > self.T:
        i-= 1
      self.x[t] += 1

## Functions

In [3]:
# moving patients around iteratively 
# local search
def simpleSearch(x, p, beta, precision, precision_limit, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results):
  for m in range(0, I-1):
    # for k in list(range(0, I))[::-1]: # [::-1]
    k = I-1
    while k >= 0:
      x_current = x.copy()
      if x_current[k] > 0: # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k+1
        next_k = (k + m + 1) % I

        x_current[k] -= 1
        x_current[next_k] += 1

        temp_results = calcResults(x_current, p, beta, precision, precision_limit, n, no_show, I, d, 0, alpha_W, alpha_I, alpha_T)
        
        if temp_results['objVal'] < results['objVal']:
          # return x_new, temp_results
          x = x_current
          results = temp_results
          k += 1
          print(x)
          print(results)

        else: # undo the previous move
          x_current[k] += 1
          x_current[next_k] -= 1
          k -= 1
        
      else:
        k -= 1
    

  return x, results

In [4]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def create_p(beta, size, precision=0.9999): # Poisson distribution 
  k = 0
  p = []

  while sum(p) < precision: # fill accurate values up to precision limit
    p.append(poisson.pmf(k, beta))
    k+=1

  while len(p) < size: # fill the rest of the values with 0
    p.append(0)
  return p, k

In [5]:
def calcExponentialLimit(mu):
  return int(max(mu+4*mu**0.5,100))

In [6]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))

def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [7]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  for k in range(limit):
    tardiness += k * p_min[I][k] # I+1
  return tardiness


def calcIdletime(I, d, tardiness, n, no_show, beta):
  return (I * d) + tardiness - (n * (1 - no_show) * beta) # I-1?


def calcWaitingtime(p_min, x, p, limit, I, n):
  w = np.zeros((I+1, n+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k] 
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= n

  return waitingtime

In [8]:

def calculateProbabilities(x, p, beta, precision, precision_limit, n, no_show, I, d):
  # count = len(p)
  count = precision_limit
  waiting = 0
  idletime = 0
  limit = calcExponentialLimit(beta*n) + 1
  p_plus = np.zeros((I+1,limit+d))
  p_min = np.zeros((I+1,limit+d))
  v = np.zeros((n+1,limit+d))
  add_v = np.zeros((n+1,limit+d)) # needs a predetermined size

#	v[k][i]=probability of having i units of work given that 
#	k patients are scheduled for the interval.
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.

  add_v[0][0] = 1
  for k in range(n+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i] 
      i += 1

  for k in range(n+1): 
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 1
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  # print(p_min[0])
  # print(p_plus[0])
  for t in range(1, I+1): # calculate p_min and p_plus iteratively 
    # Constraint 3
    for k in range(d+1):
      p_min[t][0] += p_plus[t-1][k]

    # Constraint 4
    for i in range(1,limit+1):
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

    # print(p_min[t])
    # print(p_plus[t])
  

  # Tardiness calcs
  # tardiness = calcTardiness(p_min, limit, I)

  # Idle time calcs new array of given shape and type, filled with zeros.
  # idletime = calcIdletime(I, d, tardiness, n, no_show, beta)

  # Waiting time calcs
  # waitingtime = calcWaitingtime(p_min, x, w, p, limit, I, n)

  # return p_min, waiting, idletime, tardiness
  return p_min, limit

In [9]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess

In [10]:
def calcResults(x, p, beta, precision, precision_limit, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  p_min, limit = calculateProbabilities(x, p, beta, precision, precision_limit, n, no_show, I, d)
  # p_min, waitingTime, idleTime, tardiness = calculateProbabilities(x, p, beta, precision, precision_limit, n, no_show, I, d)
  
  # Tardiness calcs
  tardiness = calcTardiness(p_min, limit, I)

  # Idle time calcs new array of given shape and type, filled with zeros.
  idletime = calcIdletime(I, d, tardiness, n, no_show, beta)

  # Waiting time calcs
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, n)


  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  # Collect into a dictionary
  results = {'p_min' : p_min, 'waitingTime' : waitingtime, 'idleTime' : idletime, 'tardiness' : tardiness, 'objVal' : objVal}
  
  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess
  
  return results

# Main Function

In [11]:
precision = 0.9999
# n = 50 # number of patients
n = 24
beta = 9 # average service time for a patient
T = 4*60 # total time
d = 5 # interval size
I = int(T/d) # number of intervals

# s = schedule(beta,T,d,n)
# s.make_initial_schedule()
# x = s.x
x = list(np.zeros(I, dtype=int)) # alternate x with all patients at the beginning
x[0] = n

size = calcExponentialLimit(beta*n)+1 # size of p has to be at least as big as the limit value here
p, precision_limit = create_p(beta, size, precision)

no_show = 0
iend = 0

alpha_I = 0.2
alpha_T = 0.4 # patient doctor centric slider
alpha_W = 0.4


res = calcResults(x, p, beta, precision, precision_limit, n, no_show, I, d, iend, alpha_W, alpha_I, alpha_T)
print(x)
print(res)

x, res = simpleSearch(x, p, beta, precision, precision_limit, n, no_show, I, d, alpha_W, alpha_I, alpha_T, res)
print(x)
print(res)

[24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
{'p_min': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.37147667e-07, 8.73948642e-10, 6.77856290e-82, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.38021615e-07, 1.86306263e-76, 3.35351273e-75, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [8.37387631e-01, 1.57239708e-02, 1.46365392e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.05336843e-01, 1.05260980e-02, 9.59048762e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.49087413e-01, 6.33561849e-03, 5.65220094e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]), 'waitingTime': 103.38047613881093, 'idleTime': 24.331692496293016, 'tardiness': 0.3316924962930292, 'objVal': 46.351205953

Currently finishing in 55 mins, with the last 20 mins being no progress and just the final iterations.
#TODO evaluation metrics bugged when there is 1 or 0 in first interval? probably due to p_min problems
#TODO search can be finished but still has to perform I**2 number of probability calculations, introduce sane limit for objval?